In [1]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append('../src')

from data import tone_analyzer

%aimport data.tone_analyzer

In [2]:
from dotenv import load_dotenv
load_dotenv()
import os
import requests
import time
import textrazor

In [3]:
from pymongo import MongoClient
import numpy as np

MONGO_PASSWORD = os.environ['MONGO_USER_PASSWORD']
CROWD_TANGLE = os.environ['CROWD_TANGLE_API_TOKEN']

config = {
  'host': '18.216.52.13:27017',
  'username': 'mongo_user',
  'password': MONGO_PASSWORD,
  'authSource': 'reddit_climate_news'
}

db = MongoClient(**config).reddit_climate_news

# Crowd Tangle

## FB Interactions

In [19]:
def get_crowd_tangle(link, ct_token):
    try:
        ct_inter_req = f'https://api.crowdtangle.com/ce/interactions?link={link}&token={ct_token}'
        inter_resp = requests.get(ct_inter_req)
        return inter_resp.json()
    except KeyboardInterrupt:
        raise
    except Exception as e:
        print(str(e))
        return { 'error': str(e)}

In [20]:
count = 0
for record in db.climate_submissions.find({'crowd_tangle': {'$exists': False}}, {'_id': 1, 'url': 1}):
    url = record['url']
    ct_res = get_crowd_tangle(url, CROWD_TANGLE)
    print(count)
    db.climate_submissions.update_one({'_id': record['_id']}, {'$set': {'crowd_tangle': ct_res}})
    count += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [26]:
codes = np.array([x['crowd_tangle']['status'] for x in db.climate_submissions.find() if 'crowd_tangle' in x])

In [33]:
np.unique(codes, return_counts=True)

(array([200, 400, 401]), array([1571,   96,   16]))

## Reddit Activity

In [39]:
def get_reddit_crowd_tangle(link, ct_token):
    try:
        ct_req = f'https://api.crowdtangle.com/ce/links?link={link}&platforms=reddit&token={ct_token}&version=3.0.27'
        resp = requests.get(ct_req)
        return resp.json()
    except KeyboardInterrupt:
        raise
    except Exception as e:
        print(str(e))

In [40]:
count = 0
for record in db.climate_submissions.find():
    url = record['url']
    ct_res = get_reddit_crowd_tangle(url, CROWD_TANGLE)
    if count % 100 == 0:
        print(count)
    if ct_res:
        db.climate_submissions.update_one({'_id': record['_id']}, {'$set': {'reddit_crowd_tangle': ct_res}})
        count += 1

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
Expecting value: line 1 column 1 (char 0)
1700
1800
1900
2000
2100
2200


# Text Razor

In [32]:
textrazor.api_key = os.environ['TEXT_RAZOR_TWO']

client = textrazor.TextRazor(extractors=["entities", "topics"])
client.set_cleanup_return_cleaned(True)

count = 0
error_count = 0

for record in db.climate_submissions.find({'text_razor': {'$exists': False}}).limit(200):
    try:
        url = record['url']
        res = client.analyze_url(url)
        if count % 100 == 0:
            print(error_count)
            print(count)
        db.climate_submissions.update_one({'_id': record['_id']}, {'$set': {'text_razor': res.json}})
        count += 1
    except KeyboardInterrupt as e:
        raise e
    except textrazor.TextRazorAnalysisException as e:
#         db.climate_submissions.update_one({'_id': record['_id']}, {'$set': {'text_razor': str(e)}})
        count += 1
        print(e)
        error_count += 1

0
0
0
100
TextRazor returned HTTP Code 401: b'{"ok":false, "time":0, "error":"You have reached your daily TextRazor request limit. Please contact support@textrazor.com for more information, or visit https://www.textrazor.com to upgrade your account."}\n\n'
TextRazor returned HTTP Code 401: b'{"ok":false, "time":0, "error":"You have reached your daily TextRazor request limit. Please contact support@textrazor.com for more information, or visit https://www.textrazor.com to upgrade your account."}\n\n'
TextRazor returned HTTP Code 401: b'{"ok":false, "time":0, "error":"You have reached your daily TextRazor request limit. Please contact support@textrazor.com for more information, or visit https://www.textrazor.com to upgrade your account."}\n\n'
TextRazor returned HTTP Code 401: b'{"ok":false, "time":0, "error":"You have reached your daily TextRazor request limit. Please contact support@textrazor.com for more information, or visit https://www.textrazor.com to upgrade your account."}\n\n'
Te

In [92]:
db.climate_submissions.find({'text_razor': {'$exists': False}}).count()

/Users/willyraedy/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


2068

In [93]:
db.climate_submissions.find({'text_razor': {'$exists': True}}).count()

/Users/willyraedy/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


211

# Tone

In [7]:
count = 0
for record in db.submissions.find({'tone_analyzer': {'$exists': False}}, {'title': 1}):
    try:
        title = record['title']
        res = tone_analyzer.get(title)
        if count % 100 == 0: 
            print(count)
        db.submissions.update_one({'_id': record['_id']}, {'$set': {'tone_analyzer': res}})
    #     time.sleep(0.2)
        count += 1
    except KeyboardInterrupt:
        raise
    except Exception as e:
        print(e)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

ERROR:root:Internal server error
Traceback (most recent call last):
  File "/Users/willyraedy/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Internal server error, Code: 500 , X-global-transaction-id: 5f36e682d5673922e5012d83b0b59b60


cannot encode object: ApiException(None), of type: <class 'ibm_cloud_sdk_core.api_exception.ApiException'>
25400
25500
25600
25700
25800
25900
26000
26100
26200
26300
26400
26500
26600
26700
26800
26900
27000
27100
27200
27300
27400
27500
27600
27700
27800
27900
28000
28100
28200
28300
28400
28500
28600
28700
28800
28900
29000
29100
29200
29300
29400
29500
29600
29700
29800
29900
30000
30100
30200
30300
30400
30500
30600
30700
30800
30900
31000
31100
31200
31300
31400
31500
31600
31700
31800
31900
32000
32100
32200
32300
32400
32500
32600
32700
32800
32900
33000
33100
33200
33300
33400
33500
33600
33700
33800
33900
34000
34100
34200
34300
34400
34500
34600
34700
34800
34900
35000
35100
35200
35300
35400
35500
35600
35700
35800
35900
36000
36100
36200
36300
36400
36500
36600
36700
36800
36900
37000
37100
37200
37300
37400
37500
37600
37700
37800
37900
38000
38100
38200
38300
38400
38500
38600
38700
38800
38900
39000
39100
39200
39300
39400
39500
39600
39700
39800
39900
40000
40100
40200

ERROR:root:Bad Gateway
Traceback (most recent call last):
  File "/Users/willyraedy/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Bad Gateway, Code: 502


cannot encode object: ApiException(None), of type: <class 'ibm_cloud_sdk_core.api_exception.ApiException'>
58400
58500
58600
58700
58800
58900
59000
59100
59200
59300
59400
59500
59600
59700
59800
59900
60000
60100
60200
60300
60400
60500
60600
60700
60800
60900
61000
61100
61200
61300
61400
61500
61600
61700
61800
61900
62000
62100
62200
62300
62400
62500
62600
62700
62800
62900
63000
63100
63200
63300
63400
63500
63600
63700
63800
63900
64000
64100
64200
64300
64400
64500
64600
64700
64800
64900
65000
65100
65200
65300
65400
65500
65600
65700
65800
65900
66000
66100
66200
66300
66400
66500
66600
66700
66800
66900


...

ERROR:root:<HTML><HEAD>
<TITLE>Service Unavailable</TITLE>
</HEAD><BODY>
<H1>Service Unavailable - Zero size object</H1>
The server is temporarily unable to service your request.  Please try again
later.<P>
Reference&#32;&#35;15&#46;8ea3db17&#46;1584556896&#46;3a5b1878
</BODY></HTML>
Traceback (most recent call last):
  File "/Users/willyraedy/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: <HTML><HEAD>
<TITLE>Service Unavailable</TITLE>
</HEAD><BODY>
<H1>Service Unavailable - Zero size object</H1>
The server is temporarily unable to service your request.  Please try again
later.<P>
Reference&#32;&#35;15&#46;8ea3db17&#46;1584556896&#46;3a5b1878
</BODY></HTML>
, Code: 503


cannot encode object: ApiException(None), of type: <class 'ibm_cloud_sdk_core.api_exception.ApiException'>
109400
109500


ERROR:root:<HTML><HEAD>
<TITLE>Service Unavailable</TITLE>
</HEAD><BODY>
<H1>Service Unavailable - Zero size object</H1>
The server is temporarily unable to service your request.  Please try again
later.<P>
Reference&#32;&#35;15&#46;107f1cb8&#46;1584557083&#46;570bced3
</BODY></HTML>
Traceback (most recent call last):
  File "/Users/willyraedy/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: <HTML><HEAD>
<TITLE>Service Unavailable</TITLE>
</HEAD><BODY>
<H1>Service Unavailable - Zero size object</H1>
The server is temporarily unable to service your request.  Please try again
later.<P>
Reference&#32;&#35;15&#46;107f1cb8&#46;1584557083&#46;570bced3
</BODY></HTML>
, Code: 503


cannot encode object: ApiException(None), of type: <class 'ibm_cloud_sdk_core.api_exception.ApiException'>
109600
109700
109800
109900
110000
110100
110200
110300
110400
110500
110600
110700
110800
110900
111000
111100
111200
111300
111400
111500
111600
111700
111800
111900
112000
112100
112200
112300
cannot encode object: ConnectTimeout(MaxRetryError("HTTPSConnectionPool(host='iam.cloud.ibm.com', port=443): Max retries exceeded with url: /identity/token (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x1027de5090>, 'Connection to iam.cloud.ibm.com timed out. (connect timeout=60)'))")), of type: <class 'requests.exceptions.ConnectTimeout'>
112400
112500
cannot encode object: ConnectTimeout(MaxRetryError("HTTPSConnectionPool(host='iam.cloud.ibm.com', port=443): Max retries exceeded with url: /identity/token (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x1027c39f90>, 'Connection to iam.cloud.ibm.com timed out. (

KeyboardInterrupt: 

# Sentiment

In [4]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [6]:
analyzer = SentimentIntensityAnalyzer()

count = 0
for record in db.submissions.find():
    title = record['title']
    res = analyzer.polarity_scores(title)['compound']
    if count % 100 == 0: 
        print(count)
    db.submissions.update_one({'_id': record['_id']}, {'$set': {'sentiment_score': res}})
    count += 1

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [50]:
db.climate_submissions.find_one()

{'_id': ObjectId('5e65d004bd71f87e9891c891'),
 'all_awardings': [],
 'allow_live_comments': False,
 'author': 'PBR--Streetgang',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_nssruw2',
 'author_patreon_flair': False,
 'author_premium': True,
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1583573838,
 'domain': 'cnn.com',
 'full_link': 'https://www.reddit.com/r/worldnews/comments/fet0fd/these_south_african_farmers_say_their_cows_can/',
 'gildings': {},
 'id': 'fet0fd',
 'is_crosspostable': True,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': False,
 'is_robot_indexable': True,
 'is_self': False,
 'is_video': False,
 'link_flair_background_color': '',
 'link_flair_richtext': [],
 'link_flair_text_color': 'dark',
 'link_flair_type': 'text',
 'locked': False,
 'media_only': False,
 'no_follow': True,
 'num_comments': 1,
 'num_cro